In [ ]:
import ipywidgets as widgets
from IPython.display import display
import json
from datetime import datetime
from functools import partial

todo_items = []

# 레이아웃 설정
layout_input = widgets.Layout(width='100%')
layout_button = widgets.Layout(width='120px', height='36px')
layout_progress = widgets.Layout(width='200px', height='20px')
layout_slider = widgets.Layout(width='200px')
layout_label = widgets.Layout(width='150px')
layout_action = widgets.Layout(width='70px', height='30px', background='lightgray')

overall_progress_bar = widgets.FloatProgress(min=0, max=100, value=0, bar_style='info', layout=layout_progress)
overall_progress_label = widgets.Label(value='(0%)')

todo_list_box = widgets.VBox(layout=widgets.Layout(border='1px solid #ccc', padding='10px'))

def update_overall_progress():
    if not todo_items:
        overall_progress_bar.value = 0
        overall_progress_label.value = "(0%)"
        return
    avg = sum(i['progress'] for i in todo_items) / len(todo_items)
    overall_progress_bar.value = avg
    overall_progress_label.value = f"({int(avg)}%)"

def refresh_list():
    rows = []
    for item in todo_items:
        title = widgets.Text(value=item['title'], layout=layout_input)
        progress_bar = widgets.FloatProgress(value=item['progress'], min=0, max=100, layout=layout_progress)
        progress_slider = widgets.IntSlider(value=item['progress'], min=0, max=100, step=5, layout=layout_slider)
        progress_col = widgets.VBox([progress_bar, progress_slider])
        due_label = widgets.Label(value=item['due'].strftime('%Y-%m-%d') if item['due'] else '-', layout=layout_label)
        priority_label = widgets.Label(value=item['priority'], layout=layout_label)
        complete_btn = widgets.Button(description='완료✅', layout=layout_action)
        delete_btn = widgets.Button(description='삭제❌', layout=layout_action)
        button_box = widgets.HBox([complete_btn, delete_btn])

        def on_slider_change(change, i=item):
            i['progress'] = change['new']
            progress_bar.value = change['new']
            update_overall_progress()

        def on_complete_clicked(b, i=item):
            i['progress'] = 100
            update_overall_progress()
            refresh_list()

        def on_delete_clicked(b, i=item):
            todo_items.remove(i)
            update_overall_progress()
            refresh_list()

        progress_slider.observe(on_slider_change, names='value')
        complete_btn.on_click(partial(on_complete_clicked, i=item))
        delete_btn.on_click(partial(on_delete_clicked, i=item))

        row = widgets.GridBox([
            widgets.Label(value='할 일'), title,
            widgets.Label(value='진행률'), progress_col,
            widgets.Label(value='입력 마감일'), due_label,
            widgets.Label(value='중요도'), priority_label,
            widgets.Label(value='작업'), button_box,
        ], layout=widgets.Layout(
            grid_template_columns="100px 1fr",
            grid_gap="4px 6px",
            margin='8px 0'
        ))
        rows.append(row)
    todo_list_box.children = tuple(rows)

title_input = widgets.Text(placeholder='할 일을 입력하세요', layout=layout_input)
progress_input = widgets.IntSlider(value=0, min=0, max=100, step=5, layout=layout_slider)
due_input = widgets.DatePicker(layout=layout_input)
priority_input = widgets.Dropdown(options=['낮음', '보통', '높음'], value='보통', layout=layout_input)
add_button = widgets.Button(description='추가', button_style='success', layout=widgets.Layout(width='120px', height='40px'))

def add_task(b):
    if title_input.value.strip():
        todo_items.append({
            'title': title_input.value.strip(),
            'progress': progress_input.value,
            'due': due_input.value,
            'priority': priority_input.value
        })
        title_input.value = ''
        progress_input.value = 0
        due_input.value = None
        priority_input.value = '보통'
        update_overall_progress()
        refresh_list()

add_button.on_click(add_task)

save_btn = widgets.Button(description='저장', button_style='info', layout=widgets.Layout(width='150px', height='36px'))
load_btn = widgets.Button(description='불러오기', button_style='warning', layout=widgets.Layout(width='150px', height='36px'))

def save_data(b):
    with open('todo_data.json', 'w') as f:
        json.dump([{ **i, 'due': i['due'].strftime('%Y-%m-%d') if i['due'] else None } for i in todo_items], f)

def load_data(b):
    try:
        with open('todo_data.json', 'r') as f:
            raw = json.load(f)
            todo_items.clear()
            for i in raw:
                todo_items.append({
                    'title': i['title'],
                    'progress': i['progress'],
                    'due': datetime.strptime(i['due'], '%Y-%m-%d').date() if i['due'] else None,
                    'priority': i.get('priority', '보통')
                })
            update_overall_progress()
            refresh_list()
    except:
        print("불러오기 실패")

save_btn.on_click(save_data)
load_btn.on_click(load_data)

header_label = widgets.Label(value='DAVID mk15 - 세로 정렬 및 연한 회갈색 버튼')

display(widgets.VBox([
    header_label,
    title_input,
    progress_input,
    due_input,
    priority_input,
    widgets.HBox([add_button], layout=widgets.Layout(justify_content='center')),
    widgets.HBox([save_btn, load_btn], layout=widgets.Layout(justify_content='center')),
    widgets.HBox([overall_progress_bar, overall_progress_label], layout=widgets.Layout(justify_content='center')),
    todo_list_box
]))

update_overall_progress()
refresh_list()
